In [ ]:
!python -m pip install -r requirements.txt

In [ ]:
import numpy as np

def gaussian_func(x: np.ndarray, theta: float, mu: float,
        sigma: float) -> float:
    return theta * np.exp(-np.power((x - mu) / sigma, 2))

def approx_func(x: np.ndarray, theta: np.ndarray, mu: np.ndarray,
        sigma: np.ndarray):
    k = len(theta)
    assert k == len(mu) == len(sigma)
    result = np.zeros(len(x))
    for i in range(k):
        result += gaussian_func(x, theta[i], mu[i], sigma[i])
    return result

def func(params: np.ndarray, *args: tuple) -> float:
    theta, mu, sigma = np.split(params, 3)
    assert len(theta) == len(mu) == len(sigma)
    x, y = args
    diff = y - approx_func(x, theta, mu, sigma)
    return np.sum(np.log(np.cosh(diff)))

def func_jac(params: np.ndarray, *args: tuple) -> np.ndarray:
    theta, mu, sigma = np.split(params, 3)
    k = len(theta)
    assert k == len(mu) == len(sigma)
    x, y = args
    result = np.zeros(3 * k)
    deriv_theta, deriv_mu, deriv_sigma = np.split(result, 3)

    tanh_diff = np.tanh(y - approx_func(x, theta, mu, sigma))
    for i in range(k):
        delta = x - mu[i]
        temp = tanh_diff * gaussian_func(x, 1.0, mu[i], sigma[i])
        deriv_theta[i] = -np.sum(temp)
        deriv_mu[i] = -2.0 * theta[i] * np.sum(temp * delta) / np.power(sigma[i], 2)
        deriv_sigma[i] = -2.0 * theta[i] * np.sum(temp * np.power(delta, 2)) / np.power(sigma[i], 3)
    return result

def func_hess(params: np.ndarray, *args: tuple) -> np.ndarray:
    theta, mu, sigma = np.split(params, 3)
    k = len(theta)
    assert k == len(mu) == len(sigma)
    x, y = args
    n = len(x)

    diff = y - approx_func(x, theta, mu, sigma)
    sech2_diff, tanh_diff = np.power(np.cosh(diff), -2), np.tanh(diff)

    result = np.zeros((3 * k, 3 * k))
    delta, delta2 = np.zeros((k, n)), np.zeros((k, n))
    gaussian = np.zeros((k, n))

    for i in range(k):
        delta[i] = x - mu[i]
        delta2[i] = np.power(delta[i], 2)
        gaussian[i] = gaussian_func(x, 1.0, mu[i], sigma[i])

    for i in range(k):
        for j in range(i, k):
            result[i, j] = np.sum(sech2_diff * gaussian[i] * gaussian[j])

    for i in range(k):
        for j in range(k):
            result[i, k + j] = theta[j] * np.sum(sech2_diff * gaussian[j] * delta[j] * gaussian[i])
            if i == j:
                result[i, k + j] -= np.sum(tanh_diff * gaussian[j] * delta[j])
            result[i, k + j] *= 2.0 / np.power(sigma[j], 2)

            result[i, 2 * k + j] = theta[j] * np.sum(sech2_diff * gaussian[j] * delta2[j] * gaussian[i])
            if i == j:
                result[i, 2 * k + j] -= np.sum(tanh_diff * gaussian[j] * delta2[j])
            result[i, 2 * k + j] *= 2.0 / np.power(sigma[j], 3)

    for i in range(k):
        for j in range(i, k):
            result[k + i, k + j] = 2.0 * theta[j] / np.power(sigma[j], 2) * np.sum(sech2_diff * gaussian[j] * delta[j] * gaussian[i] * delta[i])
            if i == j:
                result[k + i, k + j] -= np.sum(tanh_diff * (2.0 * gaussian[j] * delta2[j] / np.power(sigma[j], 2) - gaussian[j]))
            result[k + i, k + j] *= 2.0 * theta[i] / np.power(sigma[i], 2)

    for i in range(k):
        for j in range(k):
            result[k + i, 2 * k + j] = theta[j] * np.sum(sech2_diff * gaussian[j] * delta2[j] * gaussian[i] * delta[i])
            if i == j:
                result[k + i, 2 * k + j] -= np.sum(tanh_diff * gaussian[j] * np.power(delta[j], 3))
            result[k + i, 2 * k + j] *= 4.0 * theta[i] / np.power(sigma[i], 5)

    for i in range(k):
        for j in range(i, k):
            result[2 * k + i, 2 * k + j] = 2.0 * theta[j] * np.sum(sech2_diff * gaussian[j] * delta2[j] * gaussian[i] * delta2[i])
            if i == j:
                result[2 * k + i, 2 * k + j] -= np.sum(tanh_diff * gaussian[j] * np.power(delta2[j], 2))
            result[2 * k + i, 2 * k + j] *= 2.0 * theta[i] / np.power(sigma[i], 6)

    for i in range(3 * k):
        for j in range(i, 3 * k):
            result[i, j] = result[j, i]
    return result

In [ ]:
from math import cos, sin

def f(x: float) -> float:
    return 1.0 + 1.5 * x + 2.0 * cos(3.0 * x) + 3.0 * sin(7.0 * x)

start, stop, n = 0.0, 4.0, 2**7 + 1
x = np.linspace(start, stop, n)
f_vectorize = np.vectorize(f)
y = f_vectorize(x)

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(16, 9), dpi=400)
subplot = fig.add_subplot(111, facecolor='#FFFFFF')
subplot.plot(x, y, color='red', lw=2, label='y')
plt.legend()
plt.show()

In [ ]:
from numpy.random import normal

np.random.seed(567893074)
y_noise = y + normal(loc=0.0, scale=1.0, size=n)
fig = plt.figure(figsize=(16, 9), dpi=400)
subplot = fig.add_subplot(111, facecolor='#FFFFFF')
subplot.plot(x, y_noise, 'Xr', label='y_noise')
subplot.plot(x, y, color='green', lw=2, label='y')
plt.legend()
plt.show()

In [ ]:
from scipy.optimize import minimize

k = 8
params, step = np.zeros(3 * k), (stop - start) / k
theta, mu, sigma = np.split(params, 3)
for i in range(k):
    theta[i] = np.sum(x[i * k : k * (i + 1)]) / k
    mu[i] = start + (i + 0.5) * step
sigma[:] = step / 2.0
params[1] *=-1

result = minimize(fun=func, x0=params, args=(x, y_noise), method='Nelder-Mead', tol=1e-6)
print(F'Value of objective function: {result.fun}')
print(F'Description of the cause of the termination: {result.message}')
print(F'Number of evaluations of the objective functions: {result.nfev}')
print(F'Number of iterations performed by the optimizer: {result.nit}')
theta, mu, sigma = np.split(result.x, 3)
y_approx = approx_func(x, theta, mu, sigma)

fig = plt.figure(figsize=(16, 9), dpi=400)
subplot = fig.add_subplot(111, facecolor='#FFFFFF')
subplot.plot(x, y_noise, 'Xr', label='y_noise')
subplot.plot(x, y, color='green', lw=2, label='y')
subplot.plot(x, y_approx, color='blue', lw=2, label='y_approx')
plt.legend()
plt.show()

In [ ]:
result = minimize(fun=func, x0=params, args=(x, y_noise), method='BFGS', jac=func_jac, tol=1e-6)
print(F'Value of objective function: {result.fun}')
print(F'Description of the cause of the termination: {result.message}')
print(F'Number of evaluations of the objective functions: {result.nfev}')
print(F'Number of iterations performed by the optimizer: {result.nit}')
theta, mu, sigma = np.split(result.x, 3)
y_approx = approx_func(x, theta, mu, sigma)

fig = plt.figure(figsize=(16, 9), dpi=400)
subplot = fig.add_subplot(111, facecolor='#FFFFFF')
subplot.plot(x, y_noise, 'Xr', label='y_noise')
subplot.plot(x, y, color='green', lw=2, label='y')
subplot.plot(x, y_approx, color='blue', lw=2, label='y_approx')
plt.legend()
plt.show()

In [ ]:
result = minimize(fun=func, x0=params, args=(x, y_noise), method='Newton-CG', jac=func_jac, hess=func_hess)
print(F'Value of objective function: {result.fun}')
print(F'Description of the cause of the termination: {result.message}')
print(F'Number of evaluations of the objective functions: {result.nfev}')
print(F'Number of iterations performed by the optimizer: {result.nit}')
theta, mu, sigma = np.split(result.x, 3)
y_approx = approx_func(x, theta, mu, sigma)

fig = plt.figure(figsize=(16, 9), dpi=400)
subplot = fig.add_subplot(111, facecolor='#FFFFFF')
subplot.plot(x, y_noise, 'Xr', label='y_noise')
subplot.plot(x, y, color='green', lw=2, label='y')
subplot.plot(x, y_approx, color='blue', lw=2, label='y_approx')
plt.legend()
plt.show()